In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [21]:
def get_jobs(keyword, num_jobs, verbose,sleep_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="C:/Users/Nantha/Documents/Glassdoor-Job-Salary-Prediction/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    
#     url ='https://www.glassdoor.co.in/Job/jobs.htm?sc.generalKeyword="'+keyword+'"&sc.locationSeoString=us&locId=1&locT=N'
#     url = 'https://www.glassdoor.co.in/Job/jobs.htm?sc.generalKeyword="'+keyword+'"&sc.locationSeoString=san+jose&locId=1147436&locT=C'
#    seattle,WA
#     url = 'https://www.glassdoor.co.in/Job/jobs.htm?sc.generalKeyword="' + keyword + '"&sc.locationSeoString=seattle&locId=1150505&locT=C'
#     url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    #New york
#     url = 'https://www.glassdoor.co.in/Job/jobs.htm?sc.generalKeyword="'+keyword+'"&sc.locationSeoString=new+york&locId=1132348&locT=C'
    #San Jose
#     url = 'https://www.glassdoor.co.in/Job/jobs.htm?sc.generalKeyword="'+keyword+'"&sc.locationSeoString=san+jose&locId=1147436&locT=C'
    #Boston
    url = 'https://www.glassdoor.co.in/Job/jobs.htm?sc.generalKeyword="'+keyword+'"&sc.locationSeoString=boston&locId=1154532&locT=C'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(sleep_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
        except NoSuchElementException:
            pass
        
        #code to close salary estimate
        try:
            driver.find_element_by_class_name("modal_main salaryInfoContainer").click()  #clicking to the X.
        except NoSuchElementException:
            pass   

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  
            try:
                driver.find_element_by_class_name("modal_main salaryInfoContainer").click()  #clicking to the X.
            except NoSuchElementException:
                pass
            
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
            
            driver.execute_script("arguments[0].click();", job_button)
#             job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [23]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist",200, False,12)
df

Progress: 0/200
Progress: 1/200
Progress: 2/200
Progress: 3/200
Progress: 4/200
Progress: 5/200
Progress: 6/200
Progress: 7/200
Progress: 8/200
Progress: 9/200
Progress: 10/200
Progress: 11/200
Progress: 12/200
Progress: 13/200
Progress: 14/200
Progress: 15/200
Progress: 16/200
Progress: 17/200
Progress: 18/200
Progress: 19/200
Progress: 20/200
Progress: 21/200
Progress: 22/200
Progress: 23/200
Progress: 24/200
Progress: 25/200
Progress: 26/200
Progress: 27/200
Progress: 28/200
Progress: 29/200
Progress: 30/200
Progress: 31/200
Progress: 32/200
Progress: 33/200
Progress: 34/200
Progress: 35/200
Progress: 36/200
Progress: 37/200
Progress: 38/200
Progress: 39/200
Progress: 40/200
Progress: 41/200
Progress: 42/200
Progress: 43/200
Progress: 44/200
Progress: 45/200
Progress: 46/200
Progress: 47/200
Progress: 48/200
Progress: 49/200
Progress: 50/200
Progress: 51/200
Progress: 52/200
Progress: 53/200
Progress: 54/200
Progress: 55/200
Progress: 56/200
Progress: 57/200
Progress: 58/200
Progres

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist / Machine Learning Expert,$96K-$101K (Glassdoor Est.),Posting Title\nData Scientist / Machine Learni...,3.9,Novartis\n3.9,"Cambridge, MA","Basel, Switzerland",10000+ employees,1996,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,₹500+ billion (INR),-1
1,Data Scientist,$96K-$101K (Glassdoor Est.),Overview\n\n\nData Scientist\n\nLocations US-M...,4.0,Analysis Group\n4.0,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,₹10 to ₹50 billion (INR),-1
2,Data Scientist,$96K-$101K (Glassdoor Est.),Search by Keyword\nMore Options\n\nSearch by L...,4.1,ABIOMED\n4.1,"Danvers, MA","Danvers, MA",1001 to 5000 employees,1981,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,₹10 to ₹50 billion (INR),"Covidien, Boston Scientific"
3,Data Scientist,$96K-$101K (Glassdoor Est.),New England Biolabs is seeking a Data Scientis...,4.7,New England Biolabs\n4.7,"Ipswich, MA","Ipswich, MA",201 to 500 employees,1974,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,"Thermo Fisher Scientific, Enzymatics, Illumina"
4,Data Scientist,$96K-$101K (Glassdoor Est.),STR is a government research contractor specia...,4.7,Systems & Technology Research\n4.7,"Woburn, MA","Woburn, MA",201 to 500 employees,2010,Company - Private,Aerospace & Defence,Aerospace & Defence,₹10 to ₹50 billion (INR),-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Field Application Data Scientist - Senior,$58K-$99K (Glassdoor Est.),"Boston, MA / Greater New York Area, NY\n\nLook...",-1,Ontoforce NV,"Boston, MA","Ghent, Belgium",Unknown,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1
196,Managing Data Scientist/ML Engineer,$58K-$99K (Glassdoor Est.),Managing Data Scientist/ML Engineer\n\nApply N...,3.4,PA Consulting\n3.4,"Boston, MA","London, United Kingdom",1001 to 5000 employees,1943,Company - Private,Consulting,Business Services,₹10 to ₹50 billion (INR),"McKinsey & Company, Accenture, Deloitte"
197,Principal Data Scientist,$58K-$99K (Glassdoor Est.),Are you an experienced Data Scientist with exc...,3.3,Endurance International Group\n3.3,"Waltham, MA","Burlington, MA",1001 to 5000 employees,1997,Company - Public,Internet,Information Technology,₹100 to ₹500 billion (INR),-1
198,Data Scientist (Direct Candidates Only),$58K-$99K (Glassdoor Est.),The contractor will closely work with engineer...,3.7,Rangam Infotech Private Limited\n3.7,"Ayer, MA","Vadodara, India",1 to 50 employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1


In [24]:
df.to_csv("Boston_200.csv")

In [30]:
df_b = pd.read_csv("C:/Users/Nantha/Documents/Glassdoor-Job-Salary-Prediction/Boston_200.csv")
df_sj= pd.read_csv("C:/Users/Nantha/Documents/Glassdoor-Job-Salary-Prediction/San_Jose250.csv")
df_sf= pd.read_csv("C:/Users/Nantha/Documents/Glassdoor-Job-Salary-Prediction/San_Francisco_150.csv")
df_se= pd.read_csv("C:/Users/Nantha/Documents/Glassdoor-Job-Salary-Prediction/Seattle_100.csv")
df_ny= pd.read_csv("C:/Users/Nantha/Documents/Glassdoor-Job-Salary-Prediction/Ny_300.csv")
df_b.shape,df_sj.shape,df_sf.shape,df_se.shape,df_ny.shape

((200, 15), (250, 15), (150, 15), (100, 15), (300, 15))

In [48]:
dat = pd.concat([df_b,df_sj,df_sf,df_se,df_ny],axis=0).drop_duplicates().reset_index(drop=True)

In [49]:
dat.shape

(1000, 15)

In [50]:
dat.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist / Machine Learning Expert,$96K-$101K (Glassdoor Est.),Posting Title\nData Scientist / Machine Learni...,3.9,Novartis\n3.9,"Cambridge, MA","Basel, Switzerland",10000+ employees,1996,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,₹500+ billion (INR),-1
1,1,Data Scientist,$96K-$101K (Glassdoor Est.),Overview\n\n\nData Scientist\n\nLocations US-M...,4.0,Analysis Group\n4.0,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,₹10 to ₹50 billion (INR),-1
2,2,Data Scientist,$96K-$101K (Glassdoor Est.),Search by Keyword\nMore Options\n\nSearch by L...,4.1,ABIOMED\n4.1,"Danvers, MA","Danvers, MA",1001 to 5000 employees,1981,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,₹10 to ₹50 billion (INR),"Covidien, Boston Scientific"
3,3,Data Scientist,$96K-$101K (Glassdoor Est.),New England Biolabs is seeking a Data Scientis...,4.7,New England Biolabs\n4.7,"Ipswich, MA","Ipswich, MA",201 to 500 employees,1974,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,"Thermo Fisher Scientific, Enzymatics, Illumina"
4,4,Data Scientist,$96K-$101K (Glassdoor Est.),STR is a government research contractor specia...,4.7,Systems & Technology Research\n4.7,"Woburn, MA","Woburn, MA",201 to 500 employees,2010,Company - Private,Aerospace & Defence,Aerospace & Defence,₹10 to ₹50 billion (INR),-1


In [51]:
dat.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

In [52]:
dat = dat.drop('Unnamed: 0',axis=1)

In [54]:
dat.tail()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
995,Lead Data Scientist,$87K-$149K (Glassdoor Est.),Bayer is a global enterprise with core compete...,4.0,Bayer\n4.0,"Whippany, NJ","Sao Paulo, Brazil",5001 to 10000 employees,1863,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,₹500+ billion (INR),"Johnson & Johnson, Syngenta, BASF"
996,RWD Healthcare Data Scientist,$87K-$149K (Glassdoor Est.),"Real World Data, Healthcare Data Scientist\n\n...",3.6,W2O\n3.6,"Florham Park, NJ","San Francisco, CA",201 to 500 employees,2001,Company - Private,Advertising & Marketing,Business Services,₹10 to ₹50 billion (INR),-1
997,Senior Data Scientist,$87K-$149K (Glassdoor Est.),"At Northwestern Mutual, we are strong, innovat...",3.8,Northwestern Mutual\n3.8,"New York, NY","Milwaukee, WI",5001 to 10000 employees,1857,Company - Private,Brokerage Services,Finance,₹500+ billion (INR),"New York Life, Prudential"
998,Lead Data Scientist - Private Equity,$87K-$149K (Glassdoor Est.),Are you a Data Scientist with some exposure to...,-1.0,Edward Daniels Group,"New York, NY","New York, NY",1 to 50 employees,2007,Company - Private,IT Services,Information Technology,₹100 to ₹500 million (INR),-1
999,Data Scientist - OrderID,$87K-$149K (Glassdoor Est.),Job Description\nCompuForce is seeking an expe...,-1.0,CompuForce,"New York, NY","New York, NY",Unknown,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1


In [56]:
dat.to_csv("GlassdoorUSA.csv")